# 1. 芯片初始化操作

In [39]:
import time
import json
import numpy as np
import matplotlib.pyplot as plt

from pc import PS
from modules import ADC,DAC,CHIP
from cimCommand import CMD,CmdData,Packet
from cimCommand.singleCmdInfo import *

from util import plot_v_cond

In [40]:
chip=CHIP(PS(host="192.168.1.10", port = 7, delay=0.3, debug=False))
chip.set_device_cfg(deviceType=0)

Connected to 192.168.1.10:7
local ip: 192.168.1.15 local port: 61659


In [114]:
# chip.close()

# 2. 读器件

## 2.1 读初始化操作

In [85]:
chip.set_op_mode(read=True,row=True)
chip.set_dac_read_V(0.1)
chip.set_tia_gain(3)
print(chip.get_setting_info())

## 2.2 读一个器件

In [ ]:
cond,voltage = chip.read_one(row_index=0,col_index=0)
print(f"电导:{cond}us\n电压:{voltage}v")

## 2.3 读多行多列

In [ ]:
cond,voltage = chip.read(row_index=[0,1,2,3],col_index=[3,4,6,7],debug=True)
print(f"电导:{cond}us\n电压:{voltage}v")

## 2.4 其他读操作

### 2.4.1 自定义读配置

In [ ]:
chip.set_op_mode(read=True,row=True)
chip.set_dac_read_V(0.1)
chip.set_tia_gain(3)
print(chip.get_setting_info())

# 先需要reset所有的latch
chip.set_cim_reset()

# 配置行bank
chip.set_latch([2,4,253,255],row=True,value=None)
chip.set_bank([i for i in range(8)],row=True,value=0xFFFF_FFFF)

# 配置列bank
chip.set_latch([[2,4,1,3]],row=False,value=None)
chip.set_bank([i for i in range(8)],row=False,value=0xFFFF_FFFF)

# 产生读脉冲，并读出tia值
chip.generate_read_pulse()
cond,voltage = chip.get_tia_out([k for k in range(16)])

plot_v_cond(v= voltage,cond= cond)

# 3. 写器件

## 3.1 写初始化

In [ ]:
chip.set_op_mode(read=False,row=True)
chip.set_dac_write_V(0.1)
print(chip.get_setting_info())

## 3.2 写一个器件

In [ ]:
chip.write_one(row_index=0,col_index=0,pulse_width=0.1)

## 3.3 其他写操作

### 3.3.1 自定义配置

In [ ]:
chip.set_op_mode(read=False,row=True)
chip.set_dac_write_V(0.1)
print(chip.get_setting_info())

# 先需要reset所有的latch
chip.set_cim_reset()

# 配置行bank
chip.set_latch([2,4,253,255],row=True,value=None)
chip.set_bank([i for i in range(8)],row=True,value=0xFFFF_FFFF)

# 配置列bank
chip.set_latch([[2,4,1,3]],row=False,value=None)
chip.set_bank([i for i in range(8)],row=False,value=0xFFFF_FFFF)

# 产生写配置
chip.generate_write_pulse()

# 测试